# Интро и полезная литература

### Литература

- [Word2vec в картинках](https://habr.com/ru/post/446530/). Статья на хабре от 2019 года.
- [Презентация](http://www.machinelearning.ru/wiki/images/b/b3/Word2Vec.pdf) (2017 год) с примерами реализации кода на библиотеке gensim
 
Так как Word2Vec - это уже не просто единичная реализация, а группа алгоритмов для получения векторных представлений слов, то существует масса различных пакетов на эту тему. Мы будем пользоваться gensim.

- [Gensim – Руководство для начинающих](https://webdevblog.ru/gensim-rukovodstvo-dlya-nachinajushhih/). Здесь нужно изучить ряд терминов, чтобы понимать принцип работы библиотеки. Статья 2019 года.
- [A Beginner’s Guide to Word Embedding with Gensim Word2Vec Model](https://towardsdatascience.com/a-beginners-guide-to-word-embedding-with-gensim-word2vec-model-5970fa56cc92). Статья 2019 года. По ней хорошо учиться писать код.

### Термины

- **Токен** обычно означает «слово»
- **Документ** обычно может относиться как к «предложению» так и к «абзацу»
- **Корпус** обычно представляет собой «собрание документов» в виде пакета слов или как его еще называют мешка слов 

# Импорт локальных констант и функций

Вначале перейдём на уровень рутового пути репозитория
(в дальнейшем всегда пишем пути от корня репозитория)

In [1]:
cd ..

/Users/n.barsukov/WebstormProjects/ml-bug-detector


Заимпортируем всё, что нам пригодится в дальнейшем

In [2]:
from shared.helpers.word2VecPreprocessing import filter_tokens_fo_Word2Vec
from shared.consts import (
    PATH_TO_WORD2VEC_MODEL,
    FOLDER_WITH_JSONS, # папка, в которой лежат json c токенами
    TOKENIZED_JSON_NAME_PREFIX # как начинается имя json, в которых лежат токенизированные скрипты
)

Также внешние библиотеки подключим сразу

In [3]:
from os import walk
import pandas as pd
import json

from gensim.models.word2vec import Word2Vec
from multiprocessing import cpu_count
from tqdm.notebook import tqdm

# Готовим данные перед построением модели

Так как токенизация была разбита на микрозадачи (чтобы хватило мощностей компьютера), то и json у нас много теперь.

In [4]:
# filenames - Список всех файлов в интересуемой нами директории
fdirpath, dirnames, filenames = list(walk(FOLDER_WITH_JSONS))[0]

tokenized_scripts_jsons_paths = list(
    filter(
        lambda script_name: script_name.startswith(TOKENIZED_JSON_NAME_PREFIX),
        filenames
    )
)

Можно было бы пытаться объединить их все в одну, но опять же тут в зависимости от мощностей компьютера, может просто не хватить оперативки держать столько данных в памяти.

Благо word2Vec может учить постепенно: вначале на одних данных, потом "доучить" другим набором данных.

Поэтому вначале учим на первом наборе данных, а потом доучиваем оставшимися постепенно.

In [5]:
def transform_json_file_to_train_data(path_to_json_file):
    tokenized_scripts_dic = {}
    
    tokenizedScriptsJson = open(path_to_json_file)
    raw_tokenized_scripts_dictionary = json.load(tokenizedScriptsJson)

    for fileName in raw_tokenized_scripts_dictionary.keys():
        tokenized_scripts_dic[fileName] = filter_tokens_fo_Word2Vec(raw_tokenized_scripts_dictionary[fileName])
        
    return list(tokenized_scripts_dic.values())

In [6]:
first_train_json_path = f"{FOLDER_WITH_JSONS}/{tokenized_scripts_jsons_paths[0]}"

tokenized_scripts_list_of_lists = transform_json_file_to_train_data(first_train_json_path)

# Build Word2Vec model

Теперь бы нужно построить модель и настроить гиперпараметры.

[Эта статья](https://habr.com/ru/post/446530/) говорит, что embedding_size устанавливают обычно на 300.

In [7]:
word2VecModel = Word2Vec(
    # list of lists. Например: [["cat", "say", "meow"], ["dog", "say", "woof"]]
    sentences=tokenized_scripts_list_of_lists,
    
    # The number of dimensions of the embeddings and the default is 100
    size=300, 
    
    #The maximum distance between a target word and words around the target word. The default window is 5.
    window=5,
    
    # финальный размер словаря
    max_final_vocab=10000, 
    
    # игнорировать слова с частотностью ниже, чем эта (регулируется автоматически, если установлено max_final_vocab)
    min_count = 5, 
    
    workers=cpu_count(),
    
    # The training algorithm, either CBOW(0) or skip gram(1). The default training algorithm is CBOW
    sg=0
)

Смотрим, какие наиболее близкие вектора будут к функции с названием **setTimeout** (это встроенная функция в js, которая на вход принимает функцию первым аргументом, которую нужно исполнить через заданное кол-во времени в миллисекундах, которые будут переданы вторым аргументом).

Число напротив каждого слова - процент близости векторов.

In [8]:
word2VecModel.wv.most_similar('setTimeout', topn=20)

[('shouldHighlight', 0.9842419624328613),
 ('trigger', 0.9833998084068298),
 ('delMod', 0.9830868244171143),
 ('liveClassEventStorage', 0.9830531477928162),
 ('fnCtx', 0.9817550778388977),
 ('eventName', 0.9796693325042725),
 ('onComplete', 0.9792259931564331),
 ('setAnnotations', 0.9781937599182129),
 ('out', 0.9778815507888794),
 ('setWindowSize', 0.9755971431732178),
 ('launch', 0.9753134250640869),
 ('onMouseEvent', 0.9750669598579407),
 ('"focus"', 0.9746144413948059),
 ('click', 0.9745491743087769),
 ('fireEvent', 0.9745323657989502),
 ('currentChartNumber', 0.9737746119499207),
 ('synchronousProcessor', 0.973641574382782),
 ("'resize'", 0.973598301410675),
 ('"keydown"', 0.9734362363815308),
 ('getOption', 0.9732826352119446)]

Видим, что модель на маленьком кусочке данных ($\frac{1}{43}$ от всех токенов) показывает крайне странные результаты. Да, в каждом слове можно "натянуть" смысл, но это все будет не то.

Посмотрим лучше, какие будут результаты у модели, когда она "доучиться" на остальных данных.

# Resume learning

API библиотеки gensim предоставил возможность продолжать обучение модели на новых данных

In [9]:
additional_data_jsons = tokenized_scripts_jsons_paths[1:]

for jsonPathIndex in tqdm(range(len(additional_data_jsons))):
    try:
        new_data = transform_json_file_to_train_data(f"{FOLDER_WITH_JSONS}/{additional_data_jsons[jsonPathIndex]}")
    except:
        print('Не получилось расспарсить json:', additional_data_jsons[jsonPathIndex])
        continue
    
    word2VecModel.build_vocab(new_data, update=True)
    word2VecModel.train(new_data, total_examples=word2VecModel.corpus_count, epochs=word2VecModel.epochs)

In [10]:
word2VecModel.wv.most_similar('setTimeout', topn=20)

[('setInterval', 0.7144477367401123),
 ('clearTimeout', 0.669708788394928),
 ('clearInterval', 0.6687644124031067),
 ('5000', 0.5446996092796326),
 ('2000', 0.5427326560020447),
 ('timeout_id', 0.5411526560783386),
 ('clearTimer', 0.5382668972015381),
 ('timer', 0.5288143157958984),
 ('500', 0.521384596824646),
 ('300', 0.5210250616073608),
 ('doScrollCheck', 0.5196424126625061),
 ('scrollTo', 0.5178496837615967),
 ('timeout', 0.5132458209991455),
 ('delay', 0.5111770629882812),
 ('called_next', 0.5065167546272278),
 ('detectingSelectAll', 0.5061056613922119),
 ('focus', 0.5060896277427673),
 ('runs', 0.5051136016845703),
 ('$timeout', 0.504153311252594),
 ('waits', 0.5019734501838684)]

Результаты стали гораздо более впечатляющими!

- clearTimeout - обрывает запланированный вызов функции
- timeout - через какое время исполнить функцию (один из агрументов всегда)

# Сохранение и загрузка модели

In [11]:
word2VecModel.save(PATH_TO_WORD2VEC_MODEL)

In [12]:
from gensim.models.word2vec import Word2Vec
from shared.consts import PATH_TO_WORD2VEC_MODEL

word2VecModel = Word2Vec.load(PATH_TO_WORD2VEC_MODEL)

In [13]:
word2VecModel.wv.most_similar('setTimeout', topn=20)

[('setInterval', 0.7144477367401123),
 ('clearTimeout', 0.669708788394928),
 ('clearInterval', 0.6687644124031067),
 ('5000', 0.5446996092796326),
 ('2000', 0.5427326560020447),
 ('timeout_id', 0.5411526560783386),
 ('clearTimer', 0.5382668972015381),
 ('timer', 0.5288143157958984),
 ('500', 0.521384596824646),
 ('300', 0.5210250616073608),
 ('doScrollCheck', 0.5196424126625061),
 ('scrollTo', 0.5178496837615967),
 ('timeout', 0.5132458209991455),
 ('delay', 0.5111770629882812),
 ('called_next', 0.5065167546272278),
 ('detectingSelectAll', 0.5061056613922119),
 ('focus', 0.5060896277427673),
 ('runs', 0.5051136016845703),
 ('$timeout', 0.504153311252594),
 ('waits', 0.5019734501838684)]

In [14]:
print('В нашем словаре теперь', len(word2VecModel.wv.vocab), 'слов')

В нашем словаре теперь 128891 слов
